<a href="https://colab.research.google.com/github/LuisaBeccar/atenciones_md_laboral/blob/main/finochietto_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problema a resolver:** <br>
Contar la cantidad de atenciones realizada por el equipo de medicina ocupacional de Vision Medica en el Sanatorio Finochietto.<br>
En la planilla de ausentismo actuales se registr en la columna Observaciones cuando se reciben mails con avisos de ausentismo y certificados. Estos mails son respondidos y se consideran atenciones virtuales. Tambien sería atecnión virtual las llamadas telefonicas. Además se atiende de forma presencial a los colaboradores que concurren al consultorio de medicina ocupacional.<br>
<br>
Buscar segun indique atencion: **Virtual** o **Presencial**

In [47]:
# librerias
%%capture
import pandas as pd
import numpy as np
!pip install catboost
!uname
from sklearn.feature_extraction.text import CountVectorizer
#Spacy
!python -m spacy download es_core_news_sm
import spacy
sp = spacy.load('es_core_news_sm')
import es_core_news_md
nlp = es_core_news_md.load()

In [4]:
# librerias
%%capture
!python -m spacy download es_core_news_sm
! pip install -U symspellpy
import nltk # importar natural language toolkit
nltk.download('punkt')
nltk.download('stopwords') # modulo para descargar stopwords en diferentes idiomas
nltk.download('wordnet')
from nltk.corpus import stopwords
import pandas as pd
import numpy  as np
import re
import string
import plotly
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
import time
import spacy
import es_core_news_sm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from wordcloud import WordCloud
import pickle
from symspellpy import SymSpell
import pkg_resources
from symspellpy import SymSpell, Verbosity

import es_core_news_md
nlp = es_core_news_md.load()

# Paso a paso

In [50]:
# para poder usarlo con copy paste de la columna sin necesidad de descargar crear csv etc,
# este codigo agarra el string enorme que le pega y lo parte en filas de 100 palabras (p)

text = input("Seleccione la columna a analizar, copie y pegue aqui:") # str
p = 100  # Number of words per row

t = nlp(text) # spacy.tokens.doc.Doc

words = text.split()
rows = []
for i in range(0, len(words), p):
    rows.append(' '.join(words[i:i+p]))
X = pd.Series(rows) #pandas.core.series.Series
# X es la serie que resulta de esta funcion y que luego pasara a la otra funcion para analizar texto


Seleccione la columna a analizar, copie y pegue aqui:10/6 envian email el 9/06 desde sector, avisando ausencia de la colaboradora, dx laringitis, reposo 72 horas. Se cita a consultorio // 19/06 Acude con certificado original 10/6 envia certi 9/06 certif, dx sindrome gripal, reposo por 72 hs.//13/6 acude a presentar certificado virtual 10/6/2024 envia por email certif. 10/6 dx beg, reposo por 24 horas. 10/6 se presenta a consulta, al examen fisico, se encuentra con decaimiento, palidez, refiere nauseas. Presenta rx de senos paranasales y laboratorio del 7/06 con Pcr levadaJustifica retiro para atención por guardias. //11/6 envia desde sector aviso de ausencia, colaboradora concurrio a guardia, 48hs, gec. pendiente certificado. 11/6 envia cert del 10/6 dx: rinofaringitis viral, reposo 48hs. 12/06 aviso del sector de ausentismo de la colaboradora el 12/0 por cuadro gripal pendiente certificado 12/06 envia certificado con fecha del 10/06 con dx de gea con reposo por 48hs 13/6 se recibe con

In [51]:
X.info()

<class 'pandas.core.series.Series'>
RangeIndex: 29 entries, 0 to 28
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
29 non-null     object
dtypes: object(1)
memory usage: 360.0+ bytes


## data unipalabras, sin aplicar quitar stopwords

In [ ]:
# queda mas prolijo en cuentapalabras sin las stopwords asique de una aplicare elproximo codigo data1 y no este
"""
counter = CountVectorizer() # el conteo de todos los tokens unicos

X_t = counter.fit_transform(X)

# X_t.shape # 95 filas cada registro; 982 tokens
# counter.vocabulary_ # cada palabra cuantas apariciones tiene

data = pd.DataFrame(X_t.toarray().sum(axis=0), index = counter.get_feature_names_out(), columns=['freq'])
"""

## data1: conter aplicando quitar stopwords

In [54]:
spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS
counter1 = CountVectorizer(stop_words=list(spacy_stopwords))

X_t1 = counter1.fit_transform(X)  # counter_ sin nada, para tokenizar individualmente

data1 = pd.DataFrame(X_t1.toarray().sum(axis=0),
                    index = counter1.get_feature_names_out(),
                    columns=['freq'])
data1.sort_values('freq',ascending=False)

,freq
reposo,113
certificado,93
06,73
dx,62
mail,51
...,...
id,1
c5,1
c6,1
hora,1


## Lematizacion

In [7]:
%%capture
!python -m spacy download es_core_news_md

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/usr/local/lib/python3.10/dist-packages/thinc/types.py", line 25, in <module>
    from .compat import cupy, has_cupy
  File "/usr/local/lib/python3.10/dist-packages/thinc/compat.py", line 35, in <module>
    import torch
  File "/usr/local/lib/python3.10/dist-packages/torch/__init__.py", line 2059, in <module>
    from torch import (
  File "/usr/local/lib/python3.10/dist-packages/torch/multiprocessing/__init__.py", line 41, in <module>
    from .spawn import (
  File "/usr/local/lib/python3.10/dist-packages/torch/multiprocessing/spawn.py", line 4, in <module>
    import multiprocessing.connection
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 30, in <module>
    import _winapi
  File "<frozen importlib._boot

In [68]:
#t: spacy.tokens.doc.Doc
# solo alpha (sin carateres especiales ni numeros)
documento_alpha = [token for token in t if token.is_alpha]
#print(documento_alpha)

# sin stopwords
documento_alpha_sin_stopword = [token for token in documento_alpha if not token.is_stop]
#print(documento_alpha_sin_stopword)

# lematizacion
lemas = [token.lemma_ for token in documento_alpha_sin_stopword]
lemasdf = pd.DataFrame(lemas)
display("lemas", lemasdf.value_counts())

# import these modules
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()

# choose some words to be stemmed
stemas = pd.DataFrame([ps.stem(token) for token in lemas])
display("stems", stemas.value_counts())


'lemas'

,count
0,
reposo,119
certificado,98
dx,61
mail,51
hs,42
...,...
disa,1
dioxaflex,1
dilatacion,1


'stems'

,count
0,
reposo,119
certificado,98
dx,62
mail,51
hs,42
...,...
hepatologo,1
hepatograma,1
hepatico,1


## data2c aplicando sacar stopwords

In [ ]:
"""
counter2c = CountVectorizer(stop_words=list(stop_words),ngram_range = (2,2))  #an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams.

X_t2c = counter2c.fit_transform(X)

data2c = pd.DataFrame(X_t2c.toarray().sum(axis=0),
             index = counter2c.get_feature_names_out(),
             columns = ['freq']).sort_values('freq',ascending=False)
data2c
"""

"\ncounter2c = CountVectorizer(stop_words=list(stop_words),ngram_range = (2,2))  #an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams.\n\nX_t2c = counter2c.fit_transform(X)\n\ndata2c = pd.DataFrame(X_t2c.toarray().sum(axis=0),\n             index = counter2c.get_feature_names_out(),\n             columns = ['freq']).sort_values('freq',ascending=False)\ndata2c\n"

## data2 sin usar sacar stopwords

In [ ]:
"""
# para contar binomios quiero las stopwords presentes para poder buscar tipo "se presenta", "se recibe", "se comunica"

counter2 = CountVectorizer(ngram_range = (2,2))  #an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams.

X_t2 = counter2.fit_transform(X)

data2 = pd.DataFrame(X_t2.toarray().sum(axis=0),
             index = counter2.get_feature_names_out(),
             columns = ['freq']).sort_values('freq',ascending=False)
"""

## resolviendo


En finoch_oct_nov
data = 982 (monomios todos)<br>
data1 = 868 (usando quitar stopwords) <br>
data2 = 2366 (con binomios) <br>
data2c = 1783 (con binomios y usando quitar stopwords) <br>


♒ En el conteo a mano me dio: <br>
Virtuales: 93 <br>
Presenciales: 52

In [ ]:
"""
virtuales = (data2.query('index == "recibo certificado"').sum() +
             data2.query('index == "recibo certificados"').sum() +
             data2.query('index == "recibo certif"').sum() +
             data2.query('index == "se recibe"').sum() +
             data2.query('index == "envia certif"').sum() +
             data2.query('index == "envia certificado"').sum() +
             data2.query('index == "envia certificados"').sum() +
             data2.query('index == "envia constancia"').sum()+
             data2.query('index == "recibo mail"').sum() +
             data2.query('index == "recibo aviso"').sum() +
             data2.query('index == "se comunica"').sum() +
             data2.query('index == "sector informa"').sum()  +
             data2.query('index == "me comunico"').sum() +
             data1.query('index == "llamo"').sum() +
             data2.query('index == "intento comunicarme"').sum()
             )
# uso data1 sacados los stopwords, mas prolijo para los unipalabra,, pero sin sacarle stopwords (data2) para bipalabras porque necesito el "se"
"""
"""
solo para ver como hace el conteo, copiado lo que está en virtuales
print(virtuales,
      data2.query('index == "recibo certificado"').sum(),
      data2.query('index == "recibo certificados"').sum(),
      data2.query('index == "recibo certif"').sum(),
      data2.query('index == "se recibe"').sum(),
      data2.query('index == "envia certif"').sum(),
      data2.query('index == "envia certificado"').sum(),
      data2.query('index == "envia certificados"').sum(),
      data2.query('index == "envia constancia"').sum(),
      data2.query('index == "recibo mail"').sum(),
      data2.query('index == "recibo aviso"').sum(),
      data2.query('index == "se comunica"').sum(),
      data2.query('index == "sector informa"').sum(),
      data2.query('index == "me comunico"').sum(),
      data1.query('index == "llamo"').sum(),
      data2.query('index == "intento comunicarme"').sum()
      )
"""


'\nsolo para ver como hace el conteo, copiado lo que está en virtuales\nprint(virtuales,\n      data2.query(\'index == "recibo certificado"\').sum(),\n      data2.query(\'index == "recibo certificados"\').sum(),\n      data2.query(\'index == "recibo certif"\').sum(),\n      data2.query(\'index == "se recibe"\').sum(),\n      data2.query(\'index == "envia certif"\').sum(),\n      data2.query(\'index == "envia certificado"\').sum(),\n      data2.query(\'index == "envia certificados"\').sum(),\n      data2.query(\'index == "envia constancia"\').sum(),\n      data2.query(\'index == "recibo mail"\').sum(), \n      data2.query(\'index == "recibo aviso"\').sum(), \n      data2.query(\'index == "se comunica"\').sum(),\n      data2.query(\'index == "sector informa"\').sum(), \n      data2.query(\'index == "me comunico"\').sum(),\n      data1.query(\'index == "llamo"\').sum(),\n      data2.query(\'index == "intento comunicarme"\').sum()\n      )\n'

In [ ]:
"""
presenciales = (data1.query('index == "concurre"').sum() +
                 data1.query('index == "concurrio"').sum() +
                 data1.query('index == "acude"').sum() +
                 data1.query('index == "acudió"').sum() +
                 data2.query('index == "se presenta"').sum()+
                 data2.query('index == "se presentó"').sum())

# uso data1 sacados los stopwords, mas prolijo para los unipalabra,, pero sin sacarle stopwords (data2) para bipalabras porque necesito el "se"
"""
"""
solo para ver como hace el conteo, copiado lo que está en presenciales
print(presenciales,
      data1.query('index == "concurre"').sum(),
      data1.query('index == "concurrio"').sum(),
      data1.query('index == "acude"').sum(),
      data1.query('index == "acudió"').sum(),
      data2.query('index == "se presenta"').sum(),
      data2.query('index == "se presentó"').sum()
       )
"""

'\nsolo para ver como hace el conteo, copiado lo que está en presenciales\nprint(presenciales, \n      data1.query(\'index == "concurre"\').sum(), \n      data1.query(\'index == "concurrio"\').sum(),\n      data1.query(\'index == "acude"\').sum(), \n      data1.query(\'index == "acudió"\').sum(), \n      data2.query(\'index == "se presenta"\').sum(),\n      data2.query(\'index == "se presentó"\').sum()\n       )\n'

In [ ]:
"""
print( "Asistencias virtuales:", virtuales)
print("\n ---------------------------------------")
print( "Asistencias presenciales:", presenciales)
"""

Asistencias virtuales: freq    95
dtype: int64

 ---------------------------------------
Asistencias presenciales: freq    51
dtype: int64


# TODO JUNTO SIN CLASES

In [ ]:

# para poder usarlo con copy paste de la columna sin necesidad de descargar crear csv etc,
# este codigo agarra el string enorme que le pega y lo parte en filas de 100 palabras (p)



#-------------

text = input("Seleccione la columna a analizar, copie y pegue aqui:")
p = 100  # Number of words per row

words = text.split()
rows = []
for i in range(0, len(words), p):
    rows.append(' '.join(words[i:i+p]))
X = pd.Series(rows)
# X es la serie que resulta de esta funcion y que luego pasara a la otra funcion para analizar texto

#--------------------------

# counter unipalabras con quitador de stopwords
from spacy.lang.es import stop_words as es_stop_words
counter1 = CountVectorizer(stop_words= list(es_stop_words.STOP_WORDS))

X_t1 = counter1.fit_transform(X)  # counter_ sin nada, para tokenizar individualmente

data1 = pd.DataFrame(X_t1.toarray().sum(axis=0),
                    index = counter1.get_feature_names_out(),
                    columns=['freq'])


# -----------------------------------
# para contar binomios quiero las stopwords presentes para poder buscar tipo "se presenta", "se recibe", "se comunica"

counter2 = CountVectorizer(ngram_range = (2,2))  #an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams.

X_t2 = counter2.fit_transform(X)

data2 = pd.DataFrame(X_t2.toarray().sum(axis=0),
             index = counter2.get_feature_names_out(),
             columns = ['freq']).sort_values('freq',ascending=False)


# ------------------

virtuales = (data2.query('index == "recibo certificado"').sum() +
             data2.query('index == "recibo certificados"').sum() +
             data2.query('index == "recibo certif"').sum() +
             data2.query('index == "recibo email"').sum() +
             data2.query('index == "recibo mail"').sum() +
             data2.query('index == "se recibe"').sum() +
             data2.query('index == "recibo aviso"').sum() +
             data2.query('index == "sector avisa"').sum() +
             data2.query('index == "envia certif"').sum() +
             data2.query('index == "envia certificado"').sum() +
             data2.query('index == "envia certificados"').sum() +
             data2.query('index == "envia constancia"').sum()+
             data2.query('index == "envia mail"').sum()+
             data2.query('index == "envia email"').sum()+
             data2.query('index == "se comunica"').sum() +
             data2.query('index == "sector informa"').sum()  +
             data2.query('index == "me comunico"').sum() +
             data2.query('index == "se comunica"').sum() +
             data1.query('index == "llamo"').sum() +
             data1.query('index == "llama"').sum() +
             data2.query('index == "intento comunicarme"').sum()
             )
# uso data1 sacados los stopwords, mas prolijo para los unipalabra,, pero sin sacarle stopwords (data2) para bipalabras porque necesito el "se"

presenciales = (data1.query('index == "concurre"').sum() +
                 data1.query('index == "concurrio"').sum() +
                 data1.query('index == "acude"').sum() +
                 data1.query('index == "acudió"').sum() +
                 data2.query('index == "se presenta"').sum()+
                 data2.query('index == "se presentó"').sum()+
                 data1.query('index == "aporta"').sum())

# uso data1 sacados los stopwords, mas prolijo para los unipalabra,, pero sin sacarle stopwords (data2) para bipalabras porque necesito el "se"

print( "Asistencias virtuales:", virtuales)
print("\n ---------------------------------------")
print( "Asistencias presenciales:", presenciales)




Seleccione la columna a analizar, copie y pegue aqui:10/6 envian email el 9/06 desde sector, avisando ausencia de la colaboradora, dx laringitis, reposo 72 horas. Se cita a consultorio // 19/06 Acude con certificado original 10/6 envia certi 9/06 certif, dx sindrome gripal, reposo por 72 hs.//13/6 acude a presentar certificado virtual 10/6/2024 envia por email certif. 10/6 dx beg, reposo por 24 horas. 10/6 se presenta a consulta, al examen fisico, se encuentra con decaimiento, palidez, refiere nauseas. Presenta rx de senos paranasales y laboratorio del 7/06 con Pcr levadaJustifica retiro para atención por guardias. //11/6 envia desde sector aviso de ausencia, colaboradora concurrio a guardia, 48hs, gec. pendiente certificado. 11/6 envia cert del 10/6 dx: rinofaringitis viral, reposo 48hs. 12/06 aviso del sector de ausentismo de la colaboradora el 12/0 por cuadro gripal pendiente certificado 12/06 envia certificado con fecha del 10/06 con dx de gea con reposo por 48hs 13/6 se recibe con

# con Python orientado a objetos todo en 1 celda

Set Up Your Environment:

Make sure you have Python and pip installed on your machine.
Install your chosen framework:
text

In [ ]:
pip install streamlit
pip install gradio

In [ ]:
# librerias
import streamlit as st
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy

# programa
class AtencionAnalyzer:
    def __init__(self, text, stop_words):
        self.text = text
        self.stop_words = stop_words
        self.rows = self._prepare_data()
        self.data1 = None
        self.data2 = None

    def _prepare_data(self):
        """Divide el texto en filas de 100 palabras."""
        p = 100  # Número de palabras por fila
        words = self.text.split()
        rows = []
        for i in range(0, len(words), p):
            rows.append(' '.join(words[i:i+p]))
        return pd.Series(rows)

    def _count_unigrams(self):
        """Cuenta las palabras individuales (unigramas)."""
        counter1 = CountVectorizer(stop_words=self.stop_words)
        X_t1 = counter1.fit_transform(self.rows)
        self.data1 = pd.DataFrame(X_t1.toarray().sum(axis=0),
                                  index=counter1.get_feature_names_out(),
                                  columns=['freq'])

    def _count_bigrams(self):
        """Cuenta las combinaciones de dos palabras (bipalabras)."""
        counter2 = CountVectorizer(ngram_range=(2, 2))
        X_t2 = counter2.fit_transform(self.rows)
        self.data2 = pd.DataFrame(X_t2.toarray().sum(axis=0),
                                  index=counter2.get_feature_names_out(),
                                  columns=['freq']).sort_values('freq', ascending=False)

    def analyze(self):
        """Realiza el análisis de atenciones virtuales y presenciales."""
        self._count_unigrams()
        self._count_bigrams()

        virtuales = self._count_virtuales()
        presenciales = self._count_presenciales()

        print("Asistencias virtuales:", virtuales)
        print("\n---------------------------------------")
        print("Asistencias presenciales:", presenciales)

    def _count_virtuales(self):
        """Cuenta las asistencias virtuales."""
        return (self.data2.query('index == "recibo certificado"').sum() +
                self.data2.query('index == "recibo certificados"').sum() +
                self.data2.query('index == "recibo certif"').sum() +
                self.data2.query('index == "se recibe"').sum() +
                self.data2.query('index == "envia certif"').sum() +
                self.data2.query('index == "envia certificado"').sum() +
                self.data2.query('index == "envia certificados"').sum() +
                self.data2.query('index == "envia constancia"').sum() +
                self.data2.query('index == "recibo mail"').sum() +
                self.data2.query('index == "recibo aviso"').sum() +
                self.data2.query('index == "se comunica"').sum() +
                self.data2.query('index == "sector informa"').sum() +
                self.data2.query('index == "me comunico"').sum() +
                self.data1.query('index == "llamo"').sum() +
                self.data2.query('index == "intento comunicarme"').sum())

    def _count_presenciales(self):
        """Cuenta las asistencias presenciales."""
        return (self.data1.query('index == "concurre"').sum() +
                self.data1.query('index == "concurrio"').sum() +
                self.data1.query('index == "acude"').sum() +
                self.data1.query('index == "acudió"').sum() +
                self.data2.query('index == "se presenta"').sum() +
                self.data2.query('index == "se presentó"').sum())

# Uso de la clase
if __name__ == "__main__":
    from spacy.lang.es import stop_words as es_stop_words
    text = input("Seleccione la columna a analizar, copie y pegue aquí: ")
    stop_words = list(es_stop_words.STOP_WORDS)  # Define aquí tu lista de stop words
    analyzer = AtencionAnalyzer(text, stop_words)
    analyzer.analyze()


st.title('Atenciones Medicina Laboral')

text = st.text_area("Paste the text you want to analyze here:")
if st.button("Analyze"):
    if text:
        stop_words = list(nlp.Defaults.stop_words)
        analyzer = AtencionAnalyzer(text, stop_words)
        analyzer.analyze()
        # Display results accordingly, e.g.:
        st.write("Asistencias virtuales:", analyzer._count_virtuales())
        st.write("Asistencias presenciales:", analyzer._count_presenciales())
    else:
        st.warning("Please enter some text to analyze.")



ModuleNotFoundError: No module named 'streamlit'

Run Your App Locally:

Save your modified code in a file called app.py.
Run the app using:
text
Copy
streamlit run app.py
This will create a local server where you can see your app functioning.


In [ ]:
streamlit run app.py


Deploy Your App:

For deployment, you can use platforms like Streamlit Sharing or Heroku. For Streamlit Sharing, you'll need to push your code to a GitHub repository and link it to Streamlit Sharing.
Follow the respective deployment documentation for the chosen platform to go live.
Share Your App:

Once deployed, share the link to your app with others so they can use it online.
Summary
Using frameworks like Streamlit or Gradio allows you to create an online application quickly and without backend coding. These platforms handle server interactions and frontend management, letting you focus on your Python code and its functionality.

